In [7]:
# Import libraries
from __future__ import print_function
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.nn.functional import sigmoid, relu
from scipy.cluster.hierarchy import dendrogram, linkage

np.random.seed(2020)


In [9]:
# Read in Data
with open('data/sem_items_new.txt','r') as fid:
    names_items = np.array([l.strip() for l in fid.readlines()])
with open('data/sem_relations_new.txt','r') as fid:
    names_relations = np.array([l.strip() for l in fid.readlines()])
with open('data/sem_attributes_new.txt','r') as fid:
    names_attributes = np.array([l.strip() for l in fid.readlines()])

In [10]:
nobj = len(names_items)
nrel = len(names_relations)
nattributes = len(names_attributes)
print('List of items:')
print(*names_items,sep=',')
print("\nList of relations:")
print(*names_relations,sep=',')
print("\nList of attributes:")
print(*names_attributes,sep=',')

List of items:
Pine,Oak,Rose,Daisy,Robin,Canary,Sunfish,Salmon,Turtle,Crocodile

List of relations:
ISA,Is,Can,Has

List of attributes:
Living thing,Plant,Animal,Tree,Flower,Bird,Fish,Reptile,Pine,Oak,Rose,Daisy,Robin,Canary,Sunfish,Salmon,Turtle,Crocodile,Pretty,Big,Living,Green,Red,Yellow,Slow,Grow,Move,Swim,Fly,Sing,Skin,Roots,Leaves,Bark,Branch,Petals,Wings,Feathers,Gills,Scales,Backbone,Produce Eggs,Shell


In [11]:
item_display_ind = 35 # can change, just represents index of input data to show as example.

D = np.loadtxt('data/sem_data_new.txt')
input_pats = D[:,:nobj+nrel]
input_pats = torch.tensor(input_pats,dtype=torch.float)
output_pats = D[:,nobj+nrel:]
output_pats = torch.tensor(output_pats,dtype=torch.float)
N = input_pats.shape[0] # number of training patterns



input_v = input_pats[item_display_ind,:].numpy().astype('bool')
output_v = output_pats[item_display_ind,:].numpy().astype('bool')
print('Example input pattern:')
print(input_v.astype('int'))
print('Example output pattern:')
print(output_v.astype('int'))
print("")
print("Which encodes...")
print('Item ',end='')
print(names_items[input_v[:nobj]])
print('Relation ',end='')
print(names_relations[input_v[nobj:]])
print('Attributes ',end='')
print(names_attributes[output_v])

Example input pattern:
[0 0 0 0 0 0 0 0 1 0 0 0 0 1]
Example output pattern:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 1 0 1]

Which encodes...
Item ['Turtle']
Relation ['Has']
Attributes ['Skin' 'Backbone' 'Shell']


In [8]:
class Net(nn.Module):
    def __init__(self, rep_size, hidden_size):
        super(Net, self).__init__()
        # Input
        #  rep_size : number of hidden units in "Representation Layer"
        #  hidden_Size : number of hidden units in "Hidden Layer"
        #
        # TODO : YOUR CODE GOES HERE
        self.i2rep = nn.Linear(nobj,rep_size)
        self.all2h = nn.Linear(rep_size+nrel,hidden_size)
        self.h2o = nn.Linear(hidden_size,nattributes)
        self.sigmoid = torch.sigmoid

    def forward(self, x):
        # Defines forward pass for the network on input patterns x
        #
        # Input can take these two forms:
        #
        #   x: [nobj+nrel 1D Tensor], which is a single input pattern as a 1D tensor
        #      (containing both object and relation 1-hot identifier) (batch size is B=1)
        #   OR
        #   x : [B x (nobj+nrel) Tensor], which is a batch of B input patterns (one for each row)
        #
        # Output
        #   output [B x nattribute Tensor], which is the output pattern for each input pattern B on the Attribute Layer
        #   hidden [B x hidden_size Tensor], which are activations in the Hidden Layer
        #   rep [B x rep_size Tensor], which are the activations in the Representation LAyer
        x = x.view(-1,nobj+nrel) # reshape as size [B x (nobj+nrel) Tensor] if B=1
        x_item = x[:,:nobj] # input to Item Layer [B x nobj Tensor]
        x_rel = x[:,nobj:] # input to Relation Layer [B x nrel Tensor]
        
        rep = self.i2rep(x_item)
        rep = relu(rep)
        to_hidden = torch.cat((rep,x_rel),1)
        hidden = self.all2h(to_hidden)
        hidden = relu(hidden)
        output = self.h2o(hidden)
        output = self.sigmoid(output)
        
        return output, hidden, rep

In [33]:
loaded_model = torch.load('trained_model.pt')

mynet = Net(rep_size=len(loaded_model['model_state_dict']['i2rep.bias']),
            hidden_size=len(loaded_model['model_state_dict']['h2o.bias']))

mynet.load_state_dict(loaded_model['model_state_dict'])


RuntimeError: Error(s) in loading state_dict for Net:
	size mismatch for all2h.weight: copying a param with shape torch.Size([15, 12]) from checkpoint, the shape in current model is torch.Size([43, 12]).
	size mismatch for all2h.bias: copying a param with shape torch.Size([15]) from checkpoint, the shape in current model is torch.Size([43]).
	size mismatch for h2o.weight: copying a param with shape torch.Size([43, 15]) from checkpoint, the shape in current model is torch.Size([43, 43]).

In [32]:
loaded_model['model_state_dict']

OrderedDict([('i2rep.weight',
              tensor([[-1.6697e-01, -1.0548e-01, -8.5725e-02, -2.1847e-01, -3.1227e-01,
                        7.0004e-02, -3.0399e-01, -2.0083e-01, -1.5825e-01, -1.9332e-01],
                      [ 6.8134e-02, -2.7301e-01,  2.7604e-01, -2.7800e-01,  1.3054e-01,
                       -2.0587e-02,  2.1805e-01, -2.4378e-01, -1.8679e-01,  5.9163e-02],
                      [-3.1202e-01,  1.6521e-01,  2.2545e-01,  2.4024e-01, -1.9706e-01,
                       -2.2313e-01, -2.6819e-01, -6.1272e-02, -1.8869e-03, -1.0969e-01],
                      [-2.7036e-01, -2.9644e-01, -1.5001e-01, -3.0799e-01, -2.6625e-01,
                        1.7465e-02, -8.0025e-02, -2.0765e-01, -1.6567e-01, -1.7424e-01],
                      [ 5.3897e-01, -7.4094e-02, -7.4800e-01, -8.7271e-01, -1.3754e-01,
                        1.7907e-01,  1.0643e+00,  1.1945e+00, -8.3859e-01, -1.8213e-01],
                      [ 1.8797e+00,  2.3156e+00,  1.0869e+00,  1.3901e+00, -1.3932e+0